In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [14]:
from langchain_openai import ChatOpenAI
from langchain import hub
import bs4
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

In [15]:
llm = ChatOpenAI(model="gpt-4o-mini")
reg_prompt = hub.pull("rlm/rag-prompt")

In [44]:
loader = WebBaseLoader(
    web_path="https://applied-llms.org/",
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("content")
        )
    ),
)

docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = splitter.split_documents(docs)

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
#embedding = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = Chroma.from_documents(splits, embedding_function)


/Users/ray/.pyenv/versions/3.10.0/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [17]:
vectorstore

In [52]:
def check_rag_relevance(question):
    return vectorstore.similarity_search_with_score(question)
    
question = "What is weather today?"
#question = "What are author's thoughts of main concept of RAG?"
#doc_results = check_rag_relevance("RAG 에 대한 저자의 생각은 무엇인가?")
doc_results = check_rag_relevance("오늘 날씨는 어때?")
[d[1] for d in vectorstore.similarity_search_with_score(question, k=5 )]

[1.763567328453064,
 1.763567328453064,
 1.7873482704162598,
 1.7873482704162598,
 1.7903554439544678]